<h1> What to do when the data DataFest gives you isn't enough? </h1>

<h2> Find more data </h2>

Part 1 : Fighting with datasets
---------------------------------------------

    1. delimiters!
    2. character encodings!
    3. 200 year old website users (self-reported) :(
<script>
interested in learning more about how to markdown cells?: http://daringfireball.net/projects/markdown/syntax
</script>

Part 2: Developing new analyses
-----------------------------------------------------
    
    1. what are some vague areas you want to look into?
    2. what datasets exist that might provide grounds for you to test hypotheses?
    3. what questions do your datasets suggest to you?

    * responses to the above:
        * Books!
        * http://www2.informatik.uni-freiburg.de/~cziegler/BX/
        * exploration!

<h1> Part 1: Fighting with datasets

In [ ]:
#! conda info -e

In [ ]:
import os
os.listdir('../')

In [ ]:
import pandas as pd

<h3> Note: New datasets don't always load easily with the tools we used on the baseball dataset. We need to change how we load.</h3>

In [ ]:
books = pd.read_csv('../data/BX-Books.csv')

<h3> Question: what's a 'Traceback' i.e., stacktrace? </h3>

In [ ]:
%debug

<h3> Note. we can use the information from the stacktrace to infer and fix the root issues in our processes. </h3>

In [ ]:
with open('../data/BX-Books.csv', 'rb') as f:
    i=0
    for l in f.readlines():
        print (l)
        if i==3:
            break
        i+=1

<h3> Note. adding the correct separator gets us farther, but we still can't load data.

In [ ]:
# Fails with a CParserError

books = pd.read_csv('../data/BX-Books.csv', sep=';')

### Question. How impacted will our analyses be if we just throw out the lines we can't read?

In [ ]:
# Fails with a UnicodeDecodeError

bad_lines = []

with open('../data/BX-Books.csv', 'r') as f:
    print (f.readline())
    for l in f.readlines():
        if len(l.split(";")) > 8:
            bad_lines.append(l)

###Note. We need to tell python what kind of characters are in our file.

In [ ]:
bad_lines = []

with open('../data/BX-Books.csv', 'r', encoding='ISO-8859-1') as f:
    print (f.readline())
    for l in f.readlines():
        if len(l.split(";")) > 8:
            bad_lines.append(l)

In [ ]:
len(bad_lines)

In [ ]:
bad_lines[0:1]

<h3> Question. How many records are there total in my dataset?

In [ ]:
with open('../data/BX-Books.csv', 'r', encoding='ISO-8859-1') as f:
    total_records = len(f.readlines()) - 1 

In [ ]:
float(len(bad_lines))/total_records

### Note. if you really wanted to fix this problem, you would probably just use unix command line tools:
>LC_CTYPE=C && LANG=C && sed 's/\&amp;/\&/g' BX-Books.csv > BX-Books_no_semi_colon.txt

<h3> Note. Okay. Let's just sacrifice some records.</h3>

In [ ]:
books = pd.read_csv('../data/BX-Books.csv', sep=';', encoding='ISO-8859-1', error_bad_lines=False)

In [ ]:
books = pd.read_csv('../data/BX-Books.csv', sep=';', encoding='ISO-8859-1', error_bad_lines=False)
demo = pd.read_csv('../data/BX-Users.csv', sep=';', encoding='ISO-8859-1')
ratings = pd.read_csv('../data/BX-Book-Ratings.csv', sep=';', encoding='ISO-8859-1')



<img src="http://imgs.xkcd.com/comics/perl_problems.png ">

<h1> Part 2: Developing new analyses

<h2>Part 2.a : Figure out what data you have</h2>

<h3> So, now that it's loaded, what's actually in this books dataset? What questions does it suggest?

In [ ]:
books.head(3)

<h3> What's in the demo table?

In [ ]:
demo.head()

<h3> What's in the rating table?

In [ ]:
ratings.head()

<h2> Part 2.b : Write down some guiding questions
<h2>
Do people of different ages read books published in different years?
<p>
Do people of different ages rank the same book differently?

<h3>
Question. What does the age distribution look like?

In [ ]:
desc=demo.describe()

In [ ]:
desc

In [ ]:
demo.describe??

In [ ]:
demo.describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

def plot_age(df, col='Age'):
    """Makes a nice quartiles plot.
    
    Parameters
    ----------
    df : pandas.DataFrame
    col : str
        column that you want to plot quartiles for
    
    Returns
    -------
    matplotlib.axes
        plot object
    """
    plt.figure(figsize=(30,30)) # make a nice and large figure
    desc=df.describe() # get the data frame description
    ax = df[col].dropna().hist(bins=30)
    plt.vlines(desc.loc['25%'][col],0,45000, linewidth=5)
    plt.vlines(desc.loc['50%'][col],0,45000, linewidth=5)
    plt.vlines(desc.loc['75%'][col],0,45000, linewidth=5)
    
    return ax


<h3> What does a histogram of ages look like?

In [ ]:
plot_age(demo)

<h3> Note. Filter out unreasonable data

In [ ]:
demo2 = demo[demo.Age>10][demo.Age<100]

In [ ]:
plot_age(demo2)

In [ ]:
import numpy as np

demo2.Age.hist(bins=10)

plt.figure()

demo2.Age.apply(lambda x : np.log(x)).hist(bins=10)

<h3> Note. We need to merge our datasets to be able to ask the question "do people of different ages read books published in different years?" </h3>

<h3> Question. What are the common keys?</h3>

In [ ]:
demo2.head()

In [ ]:
ratings.head()

In [ ]:
books.head()

<h3> Note: specify merge key for the pandas dataframe

In [ ]:
df = demo2.merge(ratings, 
                 left_on='User-ID', 
                 right_on='User-ID')

all_dat = df.merge(books, 
                   left_on='ISBN', 
                   right_on='ISBN')

In [ ]:
all_dat.head()

<h3> Note. we can now build some age-related binned categorical variables and make some box plots comparing different age quantiles.

In [ ]:
import pandas as pd
pd.qcut??

In [ ]:
labels=['young','young-mid','mid','mid-old','old']

all_dat['age_bins'] = pd.qcut(all_dat.Age, 5, labels=labels)

### Note. let's summarize our data by categorical age variable

In [ ]:

print (np.mean(all_dat[all_dat.age_bins=='young'].Age))
print (np.mean(all_dat[all_dat.age_bins=='young-mid'].Age))
print (np.mean(all_dat[all_dat.age_bins=='mid'].Age))
print (np.mean(all_dat[all_dat.age_bins=='mid-old'].Age))
print (np.mean(all_dat[all_dat.age_bins=='old'].Age))


In [ ]:
all_dat.groupby??

<img src="http://nbviewer.jupyter.org/github/jkthompson/pyspark-pictures/blob/master/images/pyspark-page18.svg">

In [ ]:
all_dat.groupby('age_bins').Age.mean()

<h3> Note. Not free from our data woes yet ... we ended up putting a string field in a year column ...

In [ ]:
# Failes with DataError

all_dat.groupby('age_bins')['Year-Of-Publication'].mean()

<h3> Note. let's do a forcible type conversion and see if we get any more information

In [ ]:
# Fails with ValueError

all_dat['Year-Of-Publication'].astype(int)

In [ ]:
def cast(val):
    try:
        return int(val)
    except ValueError as e:
        print (e, val)
        return np.nan

In [ ]:
all_dat['Year-Of-Publication'] = all_dat['Year-Of-Publication'].apply(lambda x : cast(x))

In [ ]:
all_dat.groupby('age_bins')['Year-Of-Publication'].mean()

In [ ]:
all_dat.groupby('age_bins')['Year-Of-Publication'].std()

In [ ]:
all_dat.boxplot(column='Year-Of-Publication', by='age_bins')

<h3> Note. No effect of age on publication date of reading material.

<h2> Question. Do people of different ages rank the same book differently?</h2>

* Basically, for each book, we want an average of how each age group ranked it (use groupby for that) -- then we need some way to measure how similarly the various groups ranked books ... correlation matrix?


In [ ]:
gall_dat = all_dat.groupby(['ISBN','age_bins'])['Book-Rating'].mean()
gall_dat.head(10)

In [ ]:
pd.DataFrame(gall_dat).reset_index().head()

In [ ]:
pd.DataFrame.pivot??

In [ ]:
pivot_df = pd.DataFrame(gall_dat).reset_index().pivot('age_bins',columns='ISBN',values='Book-Rating')
pivot_df.head()

In [ ]:
len(pivot_df.T.dropna()), len(pivot_df.T)

In [ ]:
pd.DataFrame.corr??

In [ ]:
corr=pivot_df.T.dropna().corr()

corr.reindex(labels)[labels]

In [ ]:
import statsmodels.api as sm

df=np.fill_diagonal(corr.values, 0)
sm.graphics.plot_corr(corr, xnames=labels)
plt.show()


<h3> Note. maybe some age-bin structure in how pairs of bins rank books. 

In [ ]:
 all_dat.groupby(['ISBN','age_bins'])['Book-Rating'].size().hist(bins=100)

<h3> Question. Maybe different age groups rank the same authors differently? Maybe grouping by author instead of book would give us more interesting results?

In [ ]:
gall_dat = all_dat.groupby(['Book-Author','age_bins'])['Book-Rating'].mean()

gall_dat.head()
pivot_df = pd.DataFrame(gall_dat).reset_index().pivot('age_bins',columns='Book-Author',values='Book-Rating')
pivot_df.head()

len(pivot_df.T.dropna()), len(pivot_df.T)

In [ ]:
from matplotlib.pyplot import imshow
corr=pivot_df.T.dropna().corr()
corr.reindex(labels)[labels]

In [ ]:
df=np.fill_diagonal(corr.values, 0)
sm.graphics.plot_corr(corr, xnames=labels)
plt.show()

<h3> Question. Another view of the question about whether different age bins view the same books differently: do authors make up different percentages of the reading lists of typical people in each age bin?

In [ ]:
all_dat.groupby('Book-Author').size().sort(inplace=False, ascending=False).head()

In [ ]:
float(np.sum([1,2,3]))

In [ ]:

def t_val(val, series):
    """ t-value of val wrt series
    
    Parameters
    ----------
    val : int
    series : `pd.Series` or list of ints
    
    Returns
    -------
    float 
        t-value of input `val` relative to input `series`
    """
    return (val - np.mean(series))/float(np.std(series))


def percentage(val, series):
    """percentage of series represented by val
    
    Parameters
    ----------
    val : int
    series : `pd.Series` or list of ints
    
    Returns
    -------
    float 
        percentage of input `val` represents relative to sum of vals in `series`
    """
    
    return (val / float(np.sum(series)))


def author_ranker(author, rank_function):
    """applies input ranking function for author to all age_bins in df"""
    
    for age_bin, grouped in all_dat.groupby('age_bins'): # iterate on your groups
        sizes=grouped.groupby('Book-Author').size() # run on a new groupby on the grouped items
        yield {'age_bin': age_bin, author:rank_function(sizes[author], sizes)} # generator magic ... you can also just 
                                                                               # list.append
    
    
def author_plotter(author, rank_function):
    """builds a plot for author rank across all age_bins in df
    
    Parameters
    ----------
    author : str
    rank_function : types.FunctionType
    
    Returns
    -------
    matplotlib.axes
        a matplotlib plot
    """
    
    author_rankings = author_ranker(author, rank_function)
    df = pd.DataFrame(author_rankings).set_index('age_bin') # this is great, gives us rank for each author
    df = df.reindex(age_bins) # but age bins are out of order, so re-order
    
    ax = df[author].plot(legend=True) # now plot the author
    plt.title(rank_function.__name__) # add a title by 'looking inside' the function you passed in
    plt.ylabel(rank_function.__name__)
    
    return ax
    
    
    
authors=['Stephen King', 'James Patterson', 'Mary Higgins Clark', 'J. K. Rowling', 'Danielle Steel']
"""list: a set of authors we want to run stats on"""

age_bins = ['young', 'young-mid', 'mid', 'mid-old', 'old']
"""list: the age bins represented in our value datafame"""

if __name__ == '__main__':

    for author in authors:
        author_plotter(author, t_val)

    plt.figure()

    for author in authors:
        author_plotter(author, percentage)
    

In [ ]:
plt.figure(figsize=(30,30))
all_dat.groupby('Book-Author').size().sort(inplace=False, ascending=False).hist(bins=100)

In [ ]:
labels